# Importing Libraries

In [8]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
import nltk
from wordcloud import WordCloud, STOPWORDS

In [11]:
data = pd.read_csv("Tweets.csv")

# Describing data

In [12]:
data.shape

(14640, 15)

In [13]:
data.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


# Removing Everything except Text and Sentiment

In [14]:
data = data.loc[:,["text","airline_sentiment"]]

In [15]:
for k,tx in enumerate(data["text"]):
    li = " ".join([word for word in tx.split() if 'http' not in word and not word.startswith('@')and word != 'RT'])
    data.loc[k,"text"] = li

In [16]:
Y = data["airline_sentiment"].apply(lambda x: 0 if x == "negative"  else (1 if (x == "nuetral") else 2))

In [17]:
data["airline_sentiment"].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

# Text mining
1. Removing  Puntuation
2. Removing Symbols and Digits
2. Removing Stopword, 
3. Stemming/lemmatizing

In [18]:
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

for k,tx in enumerate(data["text"]):
    res = re.sub(r'[^\w\s]', '', tx)
    data.loc[k,"text"] = ''.join([i.lower() for i in res if not i.isdigit()])

In [19]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
from nltk.corpus import stopwords
stops = stopwords.words('english')

In [21]:
for k,tx in enumerate(data["text"]):
    li = " ".join([word for word in tx.split() if word not in stops])
    data.loc[k,"text"] = li

In [22]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
ltz = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [23]:
for k,tx in enumerate(data["text"]):
    li = " ".join([ltz.lemmatize(word) for word in tx.split()])
    data.loc[k,"text"] = li

In [24]:
X = data["text"]

# 

In [25]:
cv = CountVectorizer(lowercase=True, stop_words='english', min_df=2)
cvs = cv.fit_transform(X.values.astype('U'))
X_cv_df = pd.DataFrame(cvs.toarray(), columns=cv.get_feature_names())

In [26]:
X_cv_df.shape

(14640, 4974)

In [78]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_cv_df,Y,test_size = 0.1, random_state= 10)

In [28]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,Y_train)
print("Train Accuracy : " + str(model.score(X_train,Y_train)), "Test Accuracy : " + str(model.score(X_test,Y_test)))

Train Accuracy : 0.9958257437765634 Test Accuracy : 0.7971311475409836


In [29]:
model = XGBClassifier(n_estimators = 150)
model.fit(X_train,Y_train)
print("Train Accuracy : " + str(model.score(X_train,Y_train)), "Test Accuracy : " + str(model.score(X_test,Y_test)))

Train Accuracy : 0.73816029143898 Test Accuracy : 0.7199453551912568


In [30]:
model = DecisionTreeClassifier()
model.fit(X_train,Y_train)
print("Train Accuracy : " + str(model.score(X_train,Y_train)), "Test Accuracy : " + str(model.score(X_test,Y_test)))

Train Accuracy : 0.9959016393442623 Test Accuracy : 0.7342896174863388


In [31]:
from keras.models import Sequential
from keras.layers import Dense

In [79]:
def onehot(arr):
    li = np.zeros((3,len(arr)))
    for i,k in enumerate(arr):
        hot = [0,0,0]
        # print(k)
        hot[k] = 1
        li[:,i] = hot
    return li
Y_train = onehot(Y_train)
Y_test = onehot(Y_test)

In [80]:
Y_test = Y_test.transpose()
Y_train = Y_train.transpose()

In [81]:
Y_train.shape

(13176, 3)

In [82]:
X_train = np.array(X_train)
X_train.shape

(13176, 4974)

In [83]:
import tensorflow as tf
from keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

In [84]:
model = Sequential()
model.add(Dense(1000, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.9))
# model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(10,activation="relu"))
model.add(Dense(3,activation= "sigmoid"))
checkpoint = ModelCheckpoint('model_cv_nn.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')

In [85]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [86]:
history = model.fit(X_train, Y_train,validation_data = (X_test,Y_test),callbacks = [checkpoint],epochs=10, batch_size=64)

Epoch 1/10
203/206 [============================>.] - ETA: 0s - loss: 0.6299 - accuracy: 0.6626
Epoch 00001: val_accuracy improved from -inf to 0.79577, saving model to model_cv_nn.h5
206/206 [==============================] - 1s 6ms/step - loss: 0.6289 - accuracy: 0.6642 - val_loss: 0.4709 - val_accuracy: 0.7958
Epoch 2/10
205/206 [============================>.] - ETA: 0s - loss: 0.4446 - accuracy: 0.8044
Epoch 00002: val_accuracy improved from 0.79577 to 0.81762, saving model to model_cv_nn.h5
206/206 [==============================] - 1s 5ms/step - loss: 0.4451 - accuracy: 0.8045 - val_loss: 0.4091 - val_accuracy: 0.8176
Epoch 3/10
206/206 [==============================] - ETA: 0s - loss: 0.3873 - accuracy: 0.8367
Epoch 00003: val_accuracy improved from 0.81762 to 0.82377, saving model to model_cv_nn.h5
206/206 [==============================] - 2s 9ms/step - loss: 0.3873 - accuracy: 0.8367 - val_loss: 0.3947 - val_accuracy: 0.8238
Epoch 4/10
194/206 [===========================>.

# Highest Test accuarcy 82.8% percent.

In [40]:
tfv = TfidfVectorizer(lowercase=True, stop_words='english', min_df=2)
tfvs = tfv.fit_transform(X.values.astype('U'))
X_tf_df = pd.DataFrame(tfvs.toarray(), columns=tfv.get_feature_names())

In [41]:
X_tf_df.describe()

,aa,aaba,aadfw,aadv,aadvantage,aafail,aal,aano,abandoned,abc,ability,able,aboard,abq,abroad,absolute,absolutely,absurd,abt,abused,abysmal,ac,accept,acceptable,accepted,accepting,access,accident,accidentally,accommodate,accommodated,accommodating,accommodation,accomplished,according,accordingly,account,accountability,acct,accts,...,yard,yay,yea,yeah,year,yearly,yell,yelled,yelling,yep,yes,yesso,yest,yesterday,yikes,yo,york,youd,youi,youll,young,younger,youre,yousuck,youth,youve,youyou,yow,yr,yuck,yuma,yummy,yup,yvonne,yvr,yyz,zero,zone,zoom,zurich
count,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,...,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000
mean,0.005290,0.000071,0.000051,0.000049,0.000279,0.000066,0.000074,0.000049,0.000061,0.000232,0.000216,0.002821,0.000124,0.000271,0.000073,0.000669,0.000952,0.000427,0.000192,0.000069,0.000177,0.000242,0.000397,0.000664,0.000235,0.000304,0.000941,0.000197,0.000106,0.000454,0.000062,0.000234,0.000277,0.000110,0.000321,0.000137,0.001796,0.000193,0.000285,0.000051,...,0.000058,0.000393,0.000398,0.001189,0.003128,0.000066,0.000085,0.000237,0.000120,0.000515,0.006069,0.000072,0.000122,0.002524,0.000100,0.000497,0.000357,0.000485,0.000082,0.000694,0.000245,0.000060,0.003921,0.000134,0.000088,0.001441,0.000125,0.000066,0.001015,0.000067,0.000161,0.000071,0.000304,0.000054,0.000196,0.000352,0.000834,0.000262,0.000054,0.000104
std,0.042057,0.006084,0.004352,0.004223,0.010715,0.005687,0.006477,0.004223,0.005351,0.010827,0.010144,0.032248,0.007822,0.011102,0.006214,0.017728,0.022798,0.014624,0.008896,0.005986,0.009917,0.009842,0.012651,0.017067,0.010612,0.012545,0.018756,0.009941,0.006453,0.013371,0.005483,0.012012,0.011284,0.007800,0.011947,0.008598,0.025276,0.009599,0.010974,0.004346,...,0.004987,0.014824,0.014229,0.022452,0.033299,0.005693,0.005931,0.010254,0.007504,0.016134,0.052720,0.006194,0.007431,0.030511,0.009133,0.015062,0.012058,0.013637,0.007034,0.018078,0.009965,0.005111,0.038188,0.008112,0.006216,0.023532,0.010680,0.005638,0.018886,0.005717,0.009902,0.006098,0.012791,0.004618,0.008996,0.012512,0.017238,0.010058,0.004621,0.006399
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [92]:
X_train, X_test, Y_train, Y_test = train_test_split(X_tf_df,Y,test_size = 0.1, random_state= 10)

In [43]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,Y_train)
print("Train Accuracy : " + str(model.score(X_train,Y_train)), "Test Accuracy : " + str(model.score(X_test,Y_test)))

Train Accuracy : 0.9959016393442623 Test Accuracy : 0.8073770491803278


In [44]:
model = XGBClassifier(n_estimators = 150)
model.fit(X_train,Y_train)
print("Train Accuracy : " + str(model.score(X_train,Y_train)), "Test Accuracy : " + str(model.score(X_test,Y_test)))

Train Accuracy : 0.7491651487553127 Test Accuracy : 0.7288251366120219


In [45]:
model = DecisionTreeClassifier()
model.fit(X_train,Y_train)
print("Train Accuracy : " + str(model.score(X_train,Y_train)), "Test Accuracy : " + str(model.score(X_test,Y_test)))

Train Accuracy : 0.9959016393442623 Test Accuracy : 0.7527322404371585


In [93]:
def onehot(arr):
    li = np.zeros((3,len(arr)))
    for i,k in enumerate(arr):
        hot = [0,0,0]
        # print(k)
        hot[k] = 1
        li[:,i] = hot
    return li
Y_train = onehot(Y_train)
Y_test = onehot(Y_test)

In [94]:
Y_test = Y_test.transpose()
Y_train = Y_train.transpose()

In [95]:
Y_test.shape

(1464, 3)

In [96]:
X_train = np.array(X_train)
X_train.shape

(13176, 4974)

In [97]:
model = Sequential()
model.add(Dense(1000, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.9))
# model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(10,activation="relu"))
model.add(Dense(3,activation= "sigmoid"))
checkpoint = ModelCheckpoint('model_tf_nn.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')

In [98]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [99]:
history = model.fit(X_train, Y_train,validation_data = (X_test,Y_test),callbacks=[checkpoint],epochs=20, batch_size=64)

Epoch 1/20
202/206 [============================>.] - ETA: 0s - loss: 0.6904 - accuracy: 0.6231
Epoch 00001: val_accuracy improved from -inf to 0.61612, saving model to model_tf_nn.h5
206/206 [==============================] - 1s 5ms/step - loss: 0.6882 - accuracy: 0.6239 - val_loss: 0.5317 - val_accuracy: 0.6161
Epoch 2/20
197/206 [===========================>..] - ETA: 0s - loss: 0.5115 - accuracy: 0.6886
Epoch 00002: val_accuracy improved from 0.61612 to 0.82377, saving model to model_tf_nn.h5
206/206 [==============================] - 2s 9ms/step - loss: 0.5105 - accuracy: 0.6925 - val_loss: 0.4687 - val_accuracy: 0.8238
Epoch 3/20
191/206 [==========================>...] - ETA: 0s - loss: 0.3924 - accuracy: 0.8361
Epoch 00003: val_accuracy improved from 0.82377 to 0.82445, saving model to model_tf_nn.h5
206/206 [==============================] - 1s 4ms/step - loss: 0.3909 - accuracy: 0.8371 - val_loss: 0.3906 - val_accuracy: 0.8245
Epoch 4/20
197/206 [===========================>.

# Highest Accuracy Acheived 83.743

Lets Try PCA for the above vectors

In [ ]:
from sklearn.decomposition import PCA
decomp = PCA(n_components=400)
decomp.fit(X_tf_df)
X_train = decomp.transform(X_train)
X_test =  decomp.transform(X_test)

In [ ]:
model = Sequential()
model.add(Dense(200, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10,activation="relu"))
model.add(Dense(3,activation= "sigmoid"))

In [55]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [56]:
history = model.fit(X_train, Y_train,validation_data = (X_test,Y_test), epochs=20, batch_size=64)

Epoch 1/20
206/206 [==============================] - 1s 3ms/step - loss: 0.6526 - accuracy: 0.6844 - val_loss: 0.4622 - val_accuracy: 0.7883
Epoch 2/20
206/206 [==============================] - 1s 3ms/step - loss: 0.4298 - accuracy: 0.8022 - val_loss: 0.4354 - val_accuracy: 0.8019
Epoch 3/20
206/206 [==============================] - 1s 3ms/step - loss: 0.4092 - accuracy: 0.8092 - val_loss: 0.4327 - val_accuracy: 0.8033
Epoch 4/20
206/206 [==============================] - 1s 3ms/step - loss: 0.3979 - accuracy: 0.8207 - val_loss: 0.4316 - val_accuracy: 0.7978
Epoch 5/20
206/206 [==============================] - 0s 2ms/step - loss: 0.3866 - accuracy: 0.8254 - val_loss: 0.4359 - val_accuracy: 0.7978
Epoch 6/20
206/206 [==============================] - 1s 3ms/step - loss: 0.3792 - accuracy: 0.8314 - val_loss: 0.4272 - val_accuracy: 0.7999
Epoch 7/20
206/206 [==============================] - 0s 2ms/step - loss: 0.3666 - accuracy: 0.8386 - val_loss: 0.4323 - val_accuracy: 0.8067
Epoch 

This alos does not help much. Let's try our new (self trained)word2vec embeddings

#### For lstm and word embeddings based approaches please refer to the next .ipynb file



